## Langchain 

#####  What is LangChain?

LangChain is a framework that helps you build applications using Large Language Models (LLMs) like GPT, Claude, Gemini, etc.

##### Why do we need LangChain?

LLMs cannot do these things by themselves:

+ Remember past messages
+ Search your documents
+ Connect to databases
+ Call APIs
+ Use tools
+ Run multi-step reasoning
+ Build workflows

LangChain provides:

- Memory >> LLM remembers previous conversation
- Retrieval (RAG) >> LLM can fetch data from PDFs, websites, DBs
- Agents >> LLM can think, decide, and call tools
- Chains >> multi-step pipelines
- Embeddings + Vector DB >> store and search knowledge
- Tools >> like Python, search, calculators
- LangGraph >> build complex LLM workflows
- LangSmith >> evaluation and debugging

### LangChain Ecosystem


The LangChain ecosystem is made of 5 main libraries, each with a specific purpose.
Understanding these makes the entire framework easy.

##### langchain-core:
The foundation of the LangChain ecosystem.

It contains the following: 
+ Prompts (ChatPromptTemplate)
+ Runnables / Chains (LCEL pipelines)
+ Tools framework
+ Messages (HumanMessage, AIMessage)
+ Output parsers
+ Schema definitions

##### langchain-community

A big library of community-contributed integrations. Also includes the below ones.

+ Vector stores (FAISS, Pinecone, Chroma…)
+ Document loaders (PDF, Excel, websites)
+ Embedding models
+ Tools (search APIs, calculators)
+ Older chain utilities

##### langchain-openai
The OpenAI integration package >>>>  models, embeddings, chat completions. This keeps LangChain modular. It makes easy to switch model providers and also contains the following

+ ChatOpenAI
+ OpenAIEmbeddings
+ OpenAI client wrappers

This is bridge between LangChain and OpenAI/GPT models.

##### langgraph
A workflow engine for building agents and complex multi-step AI systems. This is where agents officially live now (not inside LangChain). It supports:

+ ReAct agents
+ State machines
+ Loops & conditional workflows
+ Human-in-the-loop
+ Multi-agent systems

The brain + control flow for complex AI reasoning.
LangChain = functions, LangGraph = workflows.

##### langsmith
A platform for debugging, evaluating, and monitoring LLM apps. It provides the following

+ Traces of every step in a chain/agent
+ Dataset evaluation
+ Comparison across models
+ Prompt debugging
+ Performance analytics

The observability & monitoring layer for LangChain applications.
LangSmith automatically logs chain/agent calls if enabled.

In [4]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

#### Invoke the model

In [7]:
llm = ChatOpenAI(model="gpt-4o-mini")
response = llm.invoke("What is LangChain?")
print(response.content)

LangChain is an open-source framework designed to simplify the development of applications that incorporate large language models (LLMs) and various forms of AI, including chatbots, data processing tasks, and other AI-driven solutions. It provides tools and abstractions that help developers build complex applications more easily by combining language models with external APIs, databases, and other data sources.

Key features of LangChain include:

1. **Modular Components**: LangChain offers modular components such as chains, prompts, and memory, allowing developers to construct applications with a flexible architecture.

2. **Prompt Management**: It helps manage prompt engineering, enabling developers to create, store, and use prompts efficiently.

3. **Memory Management**: LangChain includes features that allow applications to maintain context over interactions by using memory to store information between exchanges.

4. **Integration**: The framework promotes easy integration with var

#### Chain: Combine prompt >> LLM into one workflow

A Chain is a sequence of operations connected together to process input and produce output using an LLM.

In [12]:
from langchain_core.prompts import ChatPromptTemplate

In [14]:
prompt = ChatPromptTemplate.from_template("Explain {topic} in one line.")
llm = ChatOpenAI(model="gpt-4o-mini")

chain = prompt | llm 

print(chain.invoke({"topic": "Agentic AI"}).content)

Agentic AI refers to artificial intelligence systems that can operate autonomously, make decisions, and take actions in the real world based on their understanding and objectives.


In [16]:
print(chain.invoke({"topic": "Java Springboot"}).content)

Java Spring Boot is a framework that simplifies the development of stand-alone, production-ready Spring-based applications by providing built-in configurations and a range of pre-set features.


In [18]:
print(chain.invoke({"topic": "Data Stage ETL"}).content)

DataStage ETL (Extract, Transform, Load) is an IBM tool used to integrate, transform, and load data from various sources into target data warehouses or databases.


#### Tool: External function the LLM can call

In [12]:
from langchain_core.tools import tool 

@tool
def add_numbers(x: int, y: int) -> int:
    """Add two integers and return the result."""
    return x + y

result = add_numbers.invoke({"x": 5, "y": 10})
print(result)  # 15


15


#### Agent: LLM that decides which tool to use

In [14]:
import langchain
print(langchain.__version__)

1.1.3


In [15]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent

# --- Define Tool ---
@tool
def reverse_text(text: str) -> str:
    """Reverse the input text."""
    return text[::-1]

tools = [reverse_text]

# --- LLM ---
model = ChatOpenAI(model="gpt-4o-mini")

# --- Create Agent (NO prompt needed) ---
agent = create_react_agent(
    model=model,        # Correct argument name
    tools=tools
)

# --- Run it ---
result = agent.invoke({
    "messages": [
        ("user", "Reverse the word LangChain")
    ]
})

#print(result)

print(result["messages"][-1].content)

C:\Users\LotusBlue\AppData\Local\Temp\ipykernel_22584\3898682904.py:17: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(


The reversed word "LangChain" is "niahCgnaL".


#### Memory: Store and recall previous messages.

In [17]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory

# LLM
llm = ChatOpenAI(model="gpt-4o-mini")

# Prompt (IMPORTANT: include messages placeholder)
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("placeholder", "{messages}"),
    ("human", "{input}")
])

chain = prompt | llm

# Memory store
store = {}

def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

with_memory = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="messages"
)

# --- Interaction 1 ---
res1 = with_memory.invoke(
    {"input": "My name is Keerthi"},
    config={"configurable": {"session_id": "abc"}}
)
print(res1.content)  


# --- Interaction 2 ---
res2 = with_memory.invoke(
    {"input": "What is my name?"},
    config={"configurable": {"session_id": "abc"}}
)
print(res2.content)  


Hello, Keerthi! How can I assist you today?
Your name is Keerthi. How can I help you today?


#### Installation and setup 

pip install langchain-core langchain-community langchain-openai langgraph

pip install openai

In [19]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini",
    api_key=os.getenv("OPENAI_API_KEY")
)

response = llm.invoke("Hello, LangChain! Explain yourself in one line.")
print(response.content)

LangChain is a framework designed to simplify the development of applications powered by language models, enabling seamless integration of various components like tools, models, and data.
